In [ ]:
import streamlit as st
import oracledb
import pandas as pd
import os

# Configure Streamlit page
st.set_page_config(page_title="Oracle DB Query App", layout="wide")
st.title("Oracle Database Record Selector")

# Database connection configuration
@st.cache_resource
def get_db_connection():
    """Establish connection to Oracle Autonomous Database using wallet"""
    config_dir = os.path.expanduser("~/.wallet")  # Path to extracted wallet
    dsn = "your_tns_name_here"  # e.g., "indeducation_high"
    
    try:
        connection = oracledb.connect(
            user="your_username",
            password="your_password",
            dsn=dsn,
            config_dir=config_dir
        )
        return connection
    except oracledb.DatabaseError as e:
        st.error(f"Connection failed: {e}")
        return None

# Query function
def query_table(table_name, limit=100):
    """Query records from specified table"""
    conn = get_db_connection()
    if conn is None:
        return None
    
    try:
        query = f"SELECT * FROM {table_name} FETCH FIRST {limit} ROWS ONLY"
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        st.error(f"Query error: {e}")
        return None
    finally:
        conn.close()

# Sidebar configuration
st.sidebar.header("Database Configuration")
table_name = st.sidebar.text_input("Table Name", value="CUSTOMERS360")
limit = st.sidebar.slider("Number of Records", min_value=10, max_value=1000, value=100)

# Execute query and display results
if st.sidebar.button("Query Database"):
    with st.spinner("Fetching data..."):
        df = query_table(table_name, limit)
        if df is not None:
            st.success(f"Retrieved {len(df)} records")
            st.dataframe(df, use_container_width=True)
            
            # Download option
            csv = df.to_csv(index=False)
            st.download_button(
                label="Download CSV",
                data=csv,
                file_name=f"{table_name}_export.csv",
                mime="text/csv"
            )
else:
    st.info("Enter a table name and click 'Query Database' to display records")